In [ ]:
import os
from data_preprocess.Codes.Params import recode_var_dict,var_dict,read_df
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
df = read_df()
df_sent_by_author = pd.read_csv('/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/author_data.csv')
df_sent_by_author['sampWeight']=df_sent_by_author['sampWeight'].replace(' ',np.nan).replace('0',np.nan)
df_sent_by_author = df_sent_by_author.dropna(subset=['sampWeight'])
df_rand = pd.read_stata('/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Rand/randhrs1992_2018v1.dta')
def save_df(df):
    df.to_csv('/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/merge_data.csv',index=False)
def recode_dict(df,col):
    dict={}
    for var in df[col].value_counts().index:
        replace=input('for value {}, you want it replace with'.format(var))

        dict[var]=int(replace)
    return dict


# History_of_Medicaid

df0 = pd.read_csv("/data_preprocess/Data/Variables/multi_original/10_History_of_Medicaid.csv", index_col=0)
df1 = pd.read_csv("/data_preprocess/Data/Variables/multi_original/10_History_of_Medicaid_92.csv")
df2 = pd.read_csv("/data_preprocess/Data/Variables/multi_original/10_History_of_Medicaid_94.csv")

temp = pd.merge(left=df0,right=df2, left_on=['HHID','PN'],right_on=['HHID','PN'])

for index,row in df1.iterrows():
    hhid = row['HHID']
    if hhid in list(temp.HHID):
        length=len(temp.loc[temp['HHID']==hhid,])
        temp.loc[temp['HHID']==hhid,'V6604']= [row['V6604']]*length
        

temp.to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/10_History_of_Medicaid.csv")


# Never Married
draw information from the corss_wave


df_marst = pd.read_csv("/data_preprocess/Data/Variables/multi_original/9_Never_Married.csv")


columns_to_check = ['V225_x', 'W200', 'E375', 'F1071','G1158', 'HMARITAL', 'JB063', 'KB063', 'LB063']
for index,row in df_marst.iterrows():
    never_married_num = row[columns_to_check].sum()
    not_null_num = row[columns_to_check].notnull().sum()
    if never_married_num==not_null_num:
        df_marst.loc[index,'value']=1
    else:
        df_marst.loc[index,'value']=-1
    if not_null_num ==0:
        df_marst.loc[index, 'value'] = None

df_marst['value'].isnull().sum()
df_to_save = df_marst.loc[:,['HHID','PN','value']]
df_to_save.to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/9_Never_Married.csv")


#  family Income and wealth (rand)

In [ ]:

Income = df_rand.loc[:,['hhid','pn','h8itot','H9ITOT'.lower(),'H8ATOTW'.lower(),'H9ATOTW'.lower()]]

# orginal file save
# save Income
Income.loc[:,['hhid','pn','h8itot']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/non_standardized/43_Income_06.csv")
Income.loc[:,['hhid','pn','h9itot']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/non_standardized/43_Income_08.csv")

# save wealth
Income.loc[:,['hhid','pn','h8atotw']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/non_standardized/44_Wealth_06.csv")
Income.loc[:,['hhid','pn','h9atotw']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/non_standardized/44_Wealth_08.csv")

# standardise
Income.replace({0:None},inplace=True)

for column in Income.columns:
    if (column != 'hhid') & (column != 'pn'):
        Income[column]=np.log(Income.loc[:,column].astype('float'))

        Income[column] -= np.mean(Income[column], axis=0)
        Income[column] /= np.std(Income[column], axis=0)
        Income[column] = -1 * (Income[column])

# save Income
Income.loc[:,['hhid','pn','h8itot']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/43_Income_06_rev_stan.csv")
Income.loc[:,['hhid','pn','h9itot']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/43_Income_08_rev_stan.csv")

# save wealth
Income.loc[:,['hhid','pn','h8atotw']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/44_Wealth_06_rev_stan.csv")
Income.loc[:,['hhid','pn','h9atotw']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/44_Wealth_08_rev_stan.csv")



 Here we check the hist plot of income coded by us and the author 
 
 Income

In [ ]:

df['ZincomeT'].hist()
plt.show()
df_sent_by_author['Zincome'].hist()
plt.show()
print(df['ZincomeT'].describe())
print(df_sent_by_author['Zincome'].describe())

Wealth

# Lower Occupation
save to df here, rather than through the merge

In [ ]:

Occupation = df_rand.loc[:,
             ['hhid', 'pn', 'r8jcoccb', 'r9jcoccb', 'r7jcoccb', 'r6jcocc', 'r5jcocc', 'r4jcocc', 'r3jcocc', 'r2jcocc',
              'r1jcocc']]

replace_dict = {'18.office+admin support occups': '2', '01.management occupations': '1', '17.sales occups': '3',
                '09.education/training/library occs': '3', '24.transport/material moving occups': '6',
                '23.production occups': '6', '15.blding/grounds/clean/maint occups': '5', '20.construction trades': '6',
                '11.hlthcare practition/tech occs': '2', '16.personal care+service occups': '5',
                '22.install/maint/repair workrs': '5', '14.food prep+serving occs': '6',
                '07.community+social svcs occs': '5', '12.healthcare support occs': '5',
                '03.financial specialists': '2', '13.protective service occs': '4',
                '02.business oper-ns specialists': '1', '10.arts/design/entertnmt occs': '2',
                '05.architecture+engineering occs': '2', '04.computer+math occs': '2',
                '06.life/physical/socialsci occs': '2', '08.legal occups': '2', '19.farm/fish/forestry occups': '6',
                '21.extraction workers': '5', '25.military specific occups': '4', '02.prof specialty opr/tech sup': '2',
                '01.managerial specialty oper': '1', '04.clerical/admin supp': '3', '03.sales': '3',
                '09.personal svc': '5', '15.operators: transport, etc': '6', '14.operators: machine': '6',
                '10.farming/forestry/fishing': '5', '12.constr trade/extractors': '3', '11.mechanics/repair': '2',
                '16.operators: handlers, etc': '6', '13.precision production': '6', '07.svc:food prep': '5',
                '08.health svc': '5', '06.svc:protection': '4', '05.svc:prv hhld/clean/bldg svc': '5',
                '17.member of armed forces': '4'}
# for value in list(Occupation.s6jcocc.value_counts().index):
#    print(value)
#    replace = input('Input the value to replace it')
#    replace_dict[value]=replace
# print(replace_dict)
Occupation.replace(replace_dict, inplace=True)

# reverse_coded
reverse_dict = {'6': 1, '5': 2, '4': 3, '3': 4, '2': 5, '1': 6}
Occupation.replace(reverse_dict, inplace=True)


# iterate through the 8 waves to get the most recent job occupation status

for index, row in Occupation.iterrows():
    column_num = 1
    while column_num < 10:

        column = 'r{}jcocc'.format(column_num) if column_num < 7 else 'r{}jcoccb'.format(column_num)
        if ~np.isnan(float(row[column])):
            Occupation.loc[index, 'value'] = row[column]
        column_num += 1

##NOTE!!!! OCCUPATION is not merged to df through the usual Merge process, it's all done here


# standardise
Occupation.loc[:, 'value'] = Occupation.loc[:, 'value'].astype('float')
Occupation.loc[:, 'value'] -= np.mean(Occupation['value'], axis=0)
Occupation.loc[:, 'value'] /= np.std(Occupation['value'], axis=0)

# match to the df (already delete years not interviewed in 2006/2008, sample size=19193)
Occupation['hhidpn'] = [str(x) + str(y) for x, y in zip(Occupation['hhid'], Occupation['pn'])]
df['hhidpn'] = ["{:06d}".format(int(x)) + '{:03d}'.format(y) for x, y in zip(df['hhid'], df['pn'])]


for index, row in df.iterrows():
    hhidpn = row['hhidpn']
    if hhidpn in list(Occupation.hhidpn):
        value = Occupation.loc[Occupation['hhidpn'] == hhidpn, 'value'].values
        df.loc[index, 'rocc'] = value
        # print(count,value)



save_df(df)



In [ ]:
def save_df(df):
    df.to_csv('/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/merge_data.csv',index=False)
save_df(df)

# lower father occupation
df is generated in Variable_Acquire
here we reverse code it
save it first

from rand


df.loc[:,['HHID','PN','KB024M']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/non_standardized/42_Lower_Father_Occupational_Status_06_rev.csv")
df.loc[:,['HHID','PN','LB024M']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/non_standardized/42_Lower_Father_Occupational_Status_08_rev.csv")

df.loc[:,['KB024M','LB024M']].replace(reverse_dict,inplace=True)
#standardise
df.loc[:,'KB024M']=df.loc[:,'KB024M'].astype('float')
df.loc[:,'KB024M'] -= np.mean(df['KB024M'], axis=0)
df.loc[:,'KB024M'] /= np.std(df['KB024M'], axis=0)

df.loc[:,'LB024M']=df.loc[:,'LB024M'].astype('float')
df.loc[:,'LB024M'] -= np.mean(df['LB024M'], axis=0)
df.loc[:,'LB024M'] /= np.std(df['LB024M'], axis=0)

#save after standardise
df.loc[:,['HHID','PN','KB024M']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/42_Lower_Father_Occupational_Status_06_rev_stan.csv")
df.loc[:,['HHID','PN','LB024M']].to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/42_Lower_Father_Occupational_Status_08_rev_stan.csv")



In [ ]:
df_sent_by_author['fathersocc'].value_counts()

#  Alcohol
from rand

In [ ]:
Alcohol_cols = ['r8drink', 'r9drink', 'r8drinkd', 'r9drinkd', 'r8drinkn', 'r9drinkn']

Alcohol = df_rand.loc[:,['hhid','pn']+Alcohol_cols]


Alcohol.replace({'0.0 or doesnt drink':0},inplace=True)
Alcohol_drink_num = Alcohol.loc[:,['hhid','pn','r8drinkn','r9drinkn']]
Alcohol_drink_num.replace({'0.0 or doesnt drink':0},inplace=True)

# here we select the var [r8drinkn,r9drinkn]>=3 as the alcohol abuse
Alcohol_drink_num.loc[:,'r9drinkn']=Alcohol_drink_num.loc[:,'r9drinkn'].astype('float')
Alcohol_drink_num.loc[Alcohol_drink_num['r9drinkn']<3,'r9drinkn'] = -1
Alcohol_drink_num.loc[Alcohol_drink_num['r9drinkn']>=3,'r9drinkn'] = 1

Alcohol_drink_num.loc[:,'r8drinkn']=Alcohol_drink_num.loc[:,'r8drinkn'].astype('float')
Alcohol_drink_num.loc[Alcohol_drink_num['r8drinkn']<3,'r8drinkn'] = -1
Alcohol_drink_num.loc[Alcohol_drink_num['r8drinkn']>=3,'r8drinkn'] = 1


Alcohol_drink_num['r8drinkn'].value_counts()
Alcohol_drink_num['r9drinkn'].value_counts()

# save
df_to_save = Alcohol_drink_num.loc[:,['hhid','pn','r8drinkn']]
df_to_save.rename(columns ={'hhid':'HHID','pn':'PN','r8drinkn':'value', 'r9drinkn':'value'},inplace=True)
df_to_save.to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/11_Alcohol_Abuse_06.csv")

df_to_save = Alcohol_drink_num.loc[:,['hhid','pn','r9drinkn']]
df_to_save.rename(columns ={'hhid':'HHID','pn':'PN','s8jcoccb':'value', 'r9drinkn':'value'},inplace=True)
df_to_save.to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/11_Alcohol_Abuse_08.csv")


# history of smoking

In [ ]:
Smoke = df_rand.loc[:,['hhid','pn','r8smokev','r9smokev']]
Smoke.replace({'1.yes':1,'0.no':0},inplace=True)
Smoke['r9smokev'].value_counts()

# save
df_to_save = Smoke.loc[:,['hhid','pn','r8smokev']]
df_to_save.rename(columns ={'hhid':'HHID','pn':'PN','r8smokev':'value', 'r9smokev':'value'},inplace=True)
df_to_save.to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/13_History_of_Smoking_06.csv")

df_to_save = Smoke.loc[:,['hhid','pn','r9smokev']]
df_to_save.rename(columns ={'hhid':'HHID','pn':'PN','r8smokev':'value', 'r9smokev':'value'},inplace=True)
df_to_save.to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/13_History_of_Smoking_08.csv")


# current smoker

In [ ]:
smoke_now = df_rand.loc[:,['hhid','pn','r8smoken','r9smoken']]
smoke_now.replace({'1.yes':1,'0.no':0},inplace=True)
smoke_now['r9smoken'].value_counts()

# save
df_to_save = smoke_now.loc[:,['hhid','pn','r8smoken']]
df_to_save.rename(columns ={'hhid':'HHID','pn':'PN','r8smoken':'value', 'r9smoken':'value'},inplace=True)
df_to_save.to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/12_Current_Smoker_06.csv")

df_to_save = smoke_now.loc[:,['hhid','pn','r9smoken']]
df_to_save.rename(columns ={'hhid':'HHID','pn':'PN','r8smoken':'value', 'r9smoken':'value'},inplace=True)
df_to_save.to_csv("/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/12_Current_Smoker_08.csv")


# Lower Education
save to df here

# Mother and Father's education
from rand, reversed 

saved to df here

In [ ]:
columns  = ['rameduc','rafeduc']
p_education = df_rand.loc[:,['hhid','pn']+columns]
recode=recode_dict(p_education,'rameduc')
reverse_dict={'3':0,'2':1,'1':2,'None':None,'0':3}

p_education.replace(recode,inplace=True)
p_education.replace(reverse_dict,inplace=True)

p_education['rafeduc'].value_counts()

# standardise
for column in columns:
    p_education.loc[:, column] = p_education.loc[:, column].astype('float')
    p_education.loc[:, column] -= np.mean(p_education[column], axis=0)
    p_education.loc[:, column] /= np.std(p_education[column], axis=0)

# save to df
p_education['hhidpn'] = [str(x) + str(y) for x, y in zip(p_education['hhid'], p_education['pn'])]

for index,row in df.iterrows():
   hhidpn=row['hhidpn']
   if hhidpn in list(p_education.hhidpn):
       educations = p_education.loc[p_education['hhidpn']==hhidpn,['rafeduc','rameduc']]
       father_edu = educations['rafeduc'].values
       mother_edu = educations['rameduc'].values
       df.loc[index,'Zfatherseduc'] =father_edu
       df.loc[index, 'Zmotherseduc'] = mother_edu
       


save_df(df)

In [ ]:
df['fathersocc'].value_counts()

# migrantYN
from rand

saved to df here

In [ ]:

df_to_recode['rabplace'].value_counts()

# standardise
'''for column in columns:
    df_to_recode.loc[:, column] = df_to_recode.loc[:, column].astype('float')
    df_to_recode.loc[:, column] -= np.mean(df_to_recode[column], axis=0)
    df_to_recode.loc[:, column] /= np.std(df_to_recode[column], axis=0)
'''
# save to df
df_to_recode['hhidpn'] = [str(x) + str(y) for x, y in zip(df_to_recode['hhid'], df_to_recode['pn'])]

for index,row in df.iterrows():
   hhidpn=row['hhidpn']
   if hhidpn in list(df_to_recode.hhidpn):
       rows = df_to_recode.loc[df_to_recode['hhidpn']==hhidpn,columns]
       value_to_replace = rows['rabplace'].values
       df.loc[index,'migrantYN'] =value_to_replace


df['migrantYN'].isnull().sum()

df['migrantYN'].value_counts()

save_df(df)



# Black

from rand

saved to df here

In [ ]:


colname='raracem'
columns =[colname]
df_to_recode = df_rand.loc[:,['hhid','pn']+columns]
recode=recode_dict(df_to_recode,'raracem')
df_to_recode.replace(recode,inplace=True)
# reverse_dict={'3':0,'2':1,'1':2,'None':None,'0':3}
# df_to_recode.replace(reverse_dict,inplace=True)



df_to_recode[colname].value_counts()

# standardise
'''for column in columns:
    df_to_recode.loc[:, column] = df_to_recode.loc[:, column].astype('float')
    df_to_recode.loc[:, column] -= np.mean(df_to_recode[column], axis=0)
    df_to_recode.loc[:, column] /= np.std(df_to_recode[column], axis=0)
'''
# save to df
df_to_recode['hhidpn'] = [str(x) + str(y) for x, y in zip(df_to_recode['hhid'], df_to_recode['pn'])]

for index,row in df.iterrows():
   hhidpn=row['hhidpn']
   if hhidpn in list(df_to_recode.hhidpn):
       rows = df_to_recode.loc[df_to_recode['hhidpn']==hhidpn,columns]
       value_to_replace = rows[colname].values
       df.loc[index,'blackYN'] =value_to_replace


df['blackYN'].isnull().sum()

df['blackYN'].value_counts()

save_df(df)


# History of Renting ? 

saved to df here

In [ ]:

rent = pd.read_csv('/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/5_History_of_Renting.csv',index_col=0)
columns = list(rent.columns)
columns.remove('HHID')
columns.remove('PN')

for index,row in rent.iterrows():
    nan_num = row[columns].isna().sum()
    if nan_num == len(columns):
        rent.loc[index, 'value'] = None
    elif 1 in list(row[columns]):
        rent.loc[index,'value']=1
    else:
        rent.loc[index, 'value'] = -1

rent.dropna(subset=['HHID', 'PN'], inplace=True)
rent['hhidpn'] = ["{:06d}".format(int(x)) + '{:03d}'.format(int(y)) for x, y in
                            zip(rent['HHID'], rent['PN'])]

for index,row in df.iterrows():
   hhidpn=row['hhidpn']
   if hhidpn in list(rent.hhidpn):
       df.loc[index,'everrent'] = rent.loc[rent['hhidpn']==hhidpn,'value'].values



df['everrent'].isnull().sum()

df['everrent'].value_counts()

save_df(df)


# History of Medicaid

In [ ]:
medicaid = pd.read_csv('/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Variables/10_History_of_Medicaid.csv',index_col=0)
columns = list(medicaid.columns)
columns.remove('HHID')
columns.remove('PN')

for index,row in medicaid.iterrows():
    nan_num = row[columns].isna().sum()
    if nan_num == len(columns):
        medicaid.loc[index, 'value'] = None
    elif 1 in list(row[columns]):
        medicaid.loc[index,'value']=1
    else:
        medicaid.loc[index, 'value'] = -1

medicaid.dropna(subset=['HHID', 'PN'], inplace=True)
medicaid['hhidpn'] = ["{:06d}".format(int(x)) + '{:03d}'.format(int(y)) for x, y in
                            zip(medicaid['HHID'], medicaid['PN'])]

for index,row in df.iterrows():
   hhidpn=row['hhidpn']
   if hhidpn in list(medicaid.hhidpn):
       df.loc[index,'evermedicaid'] = medicaid.loc[medicaid['hhidpn']==hhidpn,'value'].values



# Death


In [ ]:

# death
df_rand = pd.read_stata('/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/Rand/randhrs1992_2018v1.dta')
death = df_rand.loc[:,['hhid','pn','RADYEAR'.lower(),'RADMONTH'.lower()]]
death['hhidpn'] = ["{:06d}".format(int(x)) + '{:03d}'.format(int(y)) for x, y in zip(death['hhid'], death['pn'])]
death['hhidpn']=death['hhidpn'].astype('float')
dfMort=df_by_us.copy()

count=0
for index,row in dfMort.iterrows():
    hhidpn=row['hhidpn']
    if hhidpn in list(death['hhidpn']):
        count+=1
        row_in_rand=death.loc[death['hhidpn']==hhidpn,]
        dfMort.loc[index,'radyear']=row_in_rand['radyear'].values[0]
        dfMort.loc[index,'radmonth']=row_in_rand['radmonth'].values[0]

dfMort['radyear'].replace({0:np.nan},inplace=True)
dfMort['radyear'].value_counts()


dfMort['death_year']=dfMort['radyear']
dfMort['death_month']=dfMort['radmonth']

dfMort['death']=[0 if np.isnan(x) else 1 for x in dfMort['radyear']]
dfMort['death'].value_counts()

dfMort['deathYR']=dfMort['radyear']+dfMort['radmonth']/12

dfMort.drop(columns=['radyear','radmonth'],inplace=True)

dfMort.to_csv('/Users/valler/Python/OX_Thesis/OX_thesis/data_preprocess/Data/merge_data_selected_author_rows_no_missing_versioin_3.csv',index=False)

